In [1]:
from tslearn.clustering import TimeSeriesKMeans
# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import os
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append("../../src")
from scipy.fft import fft
from FeaturesExtractorFromTSD import create_feature_dict_from_tsd
import numpy as np
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
#from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

c:\users\tmozo\documents\master\semester 4\computational vision\project\venv\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
paths_to_csv = {}
path_tds_angles = '../../results/big_dataset/TSD_angles'
folders = [x[0] for x in os.walk(path_tds_angles)]
folders.remove(path_tds_angles)
for folder in folders:
    video_name = folder.split('\\')[-1]
    curr_paths_to_csv = [(f[:-4], os.path.join(folder, f)) for f in listdir(folder)]
    paths_to_csv[video_name] = curr_paths_to_csv
names_of_videos = [v for v in paths_to_csv.keys() if v != 'S008C']
names_of_videos.sort(key=lambda v: v[::-1])
print(names_of_videos)

['S001C', 'S002C', 'S007C', 'S009C', 'S002I', 'S003I', 'S005I', 'S007I', 'S009I', 'S001S', 'S002S', 'S004S', 'S007S', 'S009S']


In [3]:
num_of_metrics = len(paths_to_csv[names_of_videos[0]])

In [39]:
X_train = []

In [40]:
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [41]:
def smooth_time_series(time_series, window_size):
    smoothed_series = []
    for i in range(len(time_series)):
        lower_bound = max(0, i - window_size)
        upper_bound = min(len(time_series), i + window_size + 1)
        window = time_series[lower_bound:upper_bound]
        average = sum(window) / len(window)
        smoothed_series.append(average)
    return smoothed_series

In [42]:
#fig, axs = plt.subplots(len(paths_to_csv['S005I']),len(names_of_videos), figsize=(60, 30))
for i in range(len(names_of_videos)):
    for j in [0,]:
        print(f"len X_train {len(X_train)}")
        curr_metric, curr_path = paths_to_csv[names_of_videos[i]][j]
        print(curr_metric)
        angles = pd.read_csv(curr_path)['angle'].to_list()
        #X_train = np.append(X_train, [np.array(angles)], axis=0)
        ts  = list(divide_chunks(smooth_time_series(angles,10),500))
        print(len(ts))
        X_train.extend(ts)

len X_train 0
angl_bow_mov
22
len X_train 22
angl_bow_mov
22
len X_train 44
angl_bow_mov
23
len X_train 67
angl_bow_mov
18
len X_train 85
angl_bow_mov
21
len X_train 106
angl_bow_mov
22
len X_train 128
angl_bow_mov
27
len X_train 155
angl_bow_mov
21
len X_train 176
angl_bow_mov
23
len X_train 199
angl_bow_mov
34
len X_train 233
angl_bow_mov
15
len X_train 248
angl_bow_mov
24
len X_train 272
angl_bow_mov
30
len X_train 302
angl_bow_mov
23


In [16]:
#

In [ ]:
X_train_pr = pd.DataFrame(X_train).to_numpy()
X_train_pr = np.expand_dims(X_train_pr, axis=0)


In [ ]:
kmeans = TimeSeriesKMeans(n_clusters=3, metric='euclidean')
kmeans.fit(X_train_pr)

In [ ]:
X_train_pr.shape

In [ ]:
for i in range(len(X_train)):
    X_train[i]=np.array(X_train[i])
X_train = np.array(X_train)


In [43]:
X_train_pr = pd.DataFrame(X_train)

In [44]:
X_train_pr = X_train_pr.to_numpy()

In [19]:
X_train_pr.reshape(-1, 1)

array([[117.37638639],
       [117.39018911],
       [117.40258313],
       ...,
       [         nan],
       [         nan],
       [         nan]])

In [45]:
X_train_pr.shape


(325, 500)

In [46]:
print("Soft-DTW k-means")
sdtw_km = TimeSeriesKMeans(n_clusters=6,
                           metric="softdtw",
                           verbose=False,
                           random_state=1)
y_pred = sdtw_km.fit_predict(X_train_pr)



Soft-DTW k-means


c:\users\tmozo\documents\master\semester 4\computational vision\project\venv\lib\site-packages\tslearn\barycenters\softdtw.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_ = numpy.array([to_time_series(d, remove_nans=True) for d in X_])
c:\users\tmozo\documents\master\semester 4\computational vision\project\venv\lib\site-packages\tslearn\barycenters\softdtw.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_ = numpy.array([to_time_series(d, remove_nans=True) for d in X_])


KeyboardInterrupt: 

In [ ]:
y_pred

In [ ]:
X_train_pr[y_pred == 0][0]

In [ ]:
plt.figure()
plt.subplots(figsize = (30, 30))
for yi in range(3):
    plt.subplot(10, 6, yi + 1)
    for xx in X_train_pr[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    #for xx in y.reshape(-1, 80)[y_pred == yi]:
    #    plt.plot(xx.ravel(), 'b-')
    plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Soft dtw $k$-means")

In [ ]:
print("DTW k-means")
sdtw_km = TimeSeriesKMeans(n_clusters=6,
                           metric="dtw",
                           verbose=False,
                           random_state=1)
y_pred = sdtw_km.fit_predict(X_train_pr)



In [ ]:
plt.figure()
plt.subplots(figsize = (30, 30))
for yi in range(3):
    plt.subplot(10, 6, yi + 1)
    for xx in X_train_pr[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    #for xx in y.reshape(-1, 80)[y_pred == yi]:
    #    plt.plot(xx.ravel(), 'b-')
    plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("DTW $k$-means")

In [ ]:
print("DTW k-means")
sdtw_km = TimeSeriesKMeans(n_clusters=6,
                           metric="euclidian",
                           verbose=False,
                           random_state=1)
y_pred = sdtw_km.fit_predict(X_train_pr)



In [ ]:
plt.figure()
plt.subplots(figsize = (30, 30))
for yi in range(3):
    plt.subplot(10, 6, yi + 1)
    for xx in X_train_pr[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    #for xx in y.reshape(-1, 80)[y_pred == yi]:
    #    plt.plot(xx.ravel(), 'b-')
    plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Euclidian $k$-means")